In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!cp -r "/content/drive/MyDrive/Skin_Disease_Dataset.zip" /content/

In [ ]:
!unzip -q "/content/Skin_Disease_Dataset.zip" -d /content/skin_dataset

In [ ]:
test_path  = "/content/skin_dataset/skin Disease Dataset/kaggle/test"

In [ ]:
from torchvision.datasets import ImageFolder

train_dataset = ImageFolder("/content/skin_dataset/skin Disease Dataset/kaggle/train")
print(train_dataset.classes)

['1. Enfeksiyonel', '2. Ekzama', '3. Akne', '4. Pigment', '5. Benign', '6. Malign']


In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
import numpy as np
from sklearn.utils.class_weight import compute_class_weight
import sys
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms, models
from torch.amp import autocast, GradScaler

In [ ]:
from torchvision.models import efficientnet_b1, EfficientNet_B1_Weights

from torchvision.models import mobilenet_v2, MobileNet_V2_Weights
from torchvision.models import resnet50, ResNet50_Weights
import torch
import torch.nn as nn

# MobileNetv2
model_mn = mobilenet_v2(weights="IMAGENET1K_V1")
model_mn.classifier = nn.Sequential(
    nn.Dropout(0.3),
    nn.Linear(model_mn.classifier[1].in_features, 512),
    nn.ReLU(),
    nn.Dropout(0.3),
    nn.Linear(512, 256),
    nn.ReLU(),
    nn.Dropout(0.3),
    nn.Linear(256, 6)
)
# EfficientNetb1
model_en = efficientnet_b1(weights=EfficientNet_B1_Weights.IMAGENET1K_V1)
model_en.classifier = nn.Sequential(
    nn.Dropout(0.3),
    nn.Linear(model_en.classifier[1].in_features, 512),
    nn.ReLU(),
    nn.Dropout(0.3),
    nn.Linear(512, 6)
)
# ResNet50
model_rn = resnet50(weights=ResNet50_Weights.DEFAULT)
model_rn.fc = nn.Sequential(
    nn.Dropout(0.4),
    nn.Linear(model_rn.fc.in_features, 1024),
    nn.BatchNorm1d(1024),
    nn.ReLU(inplace=True),

    nn.Dropout(0.4),
    nn.Linear(1024, 512),
    nn.BatchNorm1d(512),
    nn.ReLU(inplace=True),

    nn.Dropout(0.3),
    nn.Linear(512, 6)
)

Downloading: "https://download.pytorch.org/models/mobilenet_v2-b0353104.pth" to /root/.cache/torch/hub/checkpoints/mobilenet_v2-b0353104.pth


100%|██████████| 13.6M/13.6M [00:00<00:00, 189MB/s]


Downloading: "https://download.pytorch.org/models/efficientnet_b1_rwightman-bac287d4.pth" to /root/.cache/torch/hub/checkpoints/efficientnet_b1_rwightman-bac287d4.pth


100%|██████████| 30.1M/30.1M [00:00<00:00, 83.9MB/s]


Downloading: "https://download.pytorch.org/models/resnet50-11ad3fa6.pth" to /root/.cache/torch/hub/checkpoints/resnet50-11ad3fa6.pth


100%|██████████| 97.8M/97.8M [00:00<00:00, 120MB/s]


In [ ]:
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model_mn.load_state_dict(
    torch.load("/content/drive/My Drive/models/skin_mobile_net_v2_3.pth", map_location=DEVICE)
)
model_en.load_state_dict(
    torch.load("/content/drive/My Drive/models/efficientnet_b1_2.pth", map_location=DEVICE)
)
model_rn.load_state_dict(
    torch.load("/content/drive/My Drive/models/resnet50_2.pth", map_location=DEVICE)
)

model_mn = model_mn.to(DEVICE)
model_en = model_en.to(DEVICE)
model_rn = model_rn.to(DEVICE)

model_mn.eval()
model_en.eval()
model_rn.eval()


ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 

In [ ]:
w_mn = 0.30  # MobileNet F1 ~ 0.64
w_en = 0.30  # EfficientNet F1 ~ 0.65
w_rn = 0.40  # ResNet50 F1 ~ 0.67


In [ ]:
from torchvision import datasets, transforms
from torch.utils.data import DataLoader

test_transforms = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406],
                         [0.229, 0.224, 0.225])
])

test_dataset = datasets.ImageFolder(test_path, transform=test_transforms)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)


In [ ]:
import torch
import torch.nn.functional as F

def weighted_soft_vote(images):
    images = images.to(DEVICE)
    with torch.no_grad():
        prob_mn = F.softmax(model_mn(images), dim=1)
        prob_en = F.softmax(model_en(images), dim=1)
        prob_rn = F.softmax(model_rn(images), dim=1)

        avg_probs = w_mn*prob_mn + w_en*prob_en + w_rn*prob_rn

        final_preds = torch.argmax(avg_probs, dim=1)

    return final_preds


In [ ]:
all_preds = []
all_labels = []

for inputs, labels in test_loader:
    preds = weighted_soft_vote(inputs)
    all_preds.append(preds.cpu())
    all_labels.append(labels)

all_preds = torch.cat(all_preds)
all_labels = torch.cat(all_labels)


In [ ]:
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

print("Test Accuracy:", accuracy_score(all_labels, all_preds))
print("\nClassification Report:\n", classification_report(all_labels, all_preds, digits=4))
print("\nConfusion Matrix:\n", confusion_matrix(all_labels, all_preds))


Test Accuracy: 0.7207230142566191

Classification Report:
               precision    recall  f1-score   support

           0     0.6397    0.6533    0.6464       750
           1     0.6858    0.6549    0.6700       510
           2     0.6344    0.9161    0.7497       322
           3     0.3681    0.7794    0.5000       136
           4     0.8788    0.7245    0.7942      1361
           5     0.7750    0.7303    0.7520       849

    accuracy                         0.7207      3928
   macro avg     0.6636    0.7431    0.6854      3928
weighted avg     0.7479    0.7207    0.7269      3928


Confusion Matrix:
 [[490 101  74  60  15  10]
 [ 89 334  38  41   4   4]
 [ 10   1 295  15   1   0]
 [ 10  10  10 106   0   0]
 [114  20  32  43 986 166]
 [ 53  21  16  23 116 620]]
